In [1]:
!pip install -q datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00


In [2]:
import torch
import logging
import math

from datasets import load_dataset
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from tqdm import tqdm

# logging.basicConfig(level=logging.INFO)

In [3]:
dataset = load_dataset("winogrande", "winogrande_debiased")
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

Generating train split:   0%|          | 0/9248 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
# model.eval()
model.to('cuda')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [5]:
def predict_masked_sent(text, option_1, option_2, correct, top_k=5):
  option_1 = option_1.lower()
  option_2 = option_2.lower()
  # Tokenize input
  text = "[CLS] %s [SEP]"%text
  tokenized_text = tokenizer.tokenize(text)
  masked_index = tokenized_text.index("[MASK]")
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])
  tokens_tensor = tokens_tensor.to('cuda')

  # Predict all tokens
  with torch.no_grad():
      outputs = model(tokens_tensor)
      predictions = outputs[0]

  probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
  top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

  choice_map = dict()

  for i, pred_idx in enumerate(top_k_indices):
      predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
      token_weight = top_k_weights[i]
      if predicted_token == option_1 or predicted_token == option_2:
        # print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))
        choice_map[predicted_token] = token_weight

  if option_1 in choice_map.keys() and option_2 in choice_map.keys():
    final_choice = "1" if choice_map[option_1] >= choice_map[option_2] else "2"
  elif option_1 in choice_map.keys():
    final_choice = "1"
  elif option_2 in choice_map.keys():
    final_choice = "2"
  else:
    final_choice = "NULL"
  return (final_choice == correct, correct, choice_map)

In [6]:
ans = list()

for idx in tqdm(range(len(val_dataset["sentence"]))):
  # replace all "_" with "[MASK]"
  prompt = val_dataset["sentence"][idx].replace("_", "[MASK]")
  option1 = val_dataset["option1"][idx]
  option2 = val_dataset["option2"][idx]
  correct_ans = val_dataset["answer"][idx]
  predicted_ans = predict_masked_sent(prompt, option1, option2, correct_ans, top_k=100)
  ans.append(predicted_ans)


correct_total = [x[0] for x in ans if x[0]]

print(f"\nAccuracy: {len(correct_total) / len(ans)}")

100%|██████████| 1267/1267 [00:32<00:00, 39.30it/s]


Accuracy: 0.48224151539068666
